# インポート

In [3]:
from urllib.request import urlopen, Request
import pandas as pd
from bs4 import BeautifulSoup
import re
import time #sleep用
from tqdm.notebook import tqdm

# データ取得
### 目標
- 2024年のレース結果のテーブルをスプレイピングで取得して、Pythonで扱えるようにする

### ステップ
1. カレンダーのページから、開催日一覧を取得（スプレイピング）
    1. 2024年1月の開催であれば：https://race.netkeiba.com/top/calendar.html?year=2024&month=1
2. 開催ページから、レースid一覧を取得
    1. 2024年1月6日の開催であれば：https://race.netkeiba.com/top/race_list.html?kaisai_date=20240106
3. レース結果ページから、レース結果テーブル一覧を取得

<dl>
    <dt>・（用語の解説）スクレイピングとは</dt>
    <dd>プログラムによって、Webページから自動的にデータを取得すること</dd>
</dl>

In [9]:
from urllib.request import urlopen, Request

url = "https://db.netkeiba.com/race/202406010105/"
headers = {"User-Agent": "Mozilla/5.0"} # netkeiba.comの仕様変更により、user-agentの設定が必要になった
request = Request(url, headers=headers)
html = urlopen(request).read()

↓取得したhtmlの内容を表示してみる

In [10]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>3\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32024\xc7\xaf1\xb7\xee6\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\

pandasを使って、htmlの中身を解析する<br>
・pandasはインストールが必要<br>
・あとlxmlってやつもインストール必要

In [11]:
import pandas as pd

#pd.read_htmlで、htmlの中のtableタグが取得される
#tableタグが複数あれば、すべて取得される
pd.read_html(html)

[    着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気  \
 0     1    8   15   ミッキーラッキー  牡3  57   キング  1:59.8    NaN    2.8    1   
 1     2    1    2  マイネルモメンタム  牡3  57  石川裕紀  1:59.8    アタマ    7.3    4   
 2     3    4    7  ウイントレメンデス  牡3  57  横山武史  1:59.8     ハナ    4.4    3   
 3     4    5   10  ウォータースケイプ  牡3  57  戸崎圭太  2:00.3      3    4.2    2   
 4     5    8   16      ハイラント  牡3  57  北村宏司  2:00.4     クビ  306.7   11   
 5     6    6   12        エラン  牡3  57  ピーヒュ  2:00.5    3/4   66.9    9   
 6     7    1    1  ジーティーオウジャ  牡3  57  キングス  2:00.6     クビ   34.3    8   
 7     8    3    5    レディマキシマ  牝3  55  丹内祐次  2:00.7    1/2   20.6    7   
 8     9    7   13   イモータルバード  牡3  57  津村明秀  2:01.0      2   19.8    6   
 9    10    3    6  フェスティヴカレン  牝3  55  江田照男  2:01.3  1.3/4  489.1   15   
 10   11    2    4      ネクタール  牝3  52  佐藤翔馬  2:01.3     クビ  501.7   16   
 11   12    2    3   マイネルガンナー  牡3  57  柴田大知  2:01.3     クビ  162.1   10   
 12   13    7   14   フォローウィンド  牡3  57 

In [12]:
#０番目を指定することで、最初のtableタグのみ取得
pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,8,15,ミッキーラッキー,牡3,57,キング,1:59.8,NaN,2.8,1,550(+4),[東] 堀宣行
1,2,1,2,マイネルモメンタム,牡3,57,石川裕紀,1:59.8,アタマ,7.3,4,470(+2),[東] 相沢郁
2,3,4,7,ウイントレメンデス,牡3,57,横山武史,1:59.8,ハナ,4.4,3,480(+4),[東] 鈴木伸尋
3,4,5,10,ウォータースケイプ,牡3,57,戸崎圭太,2:00.3,3,4.2,2,554(-2),[東] 宮田敬介
4,5,8,16,ハイラント,牡3,57,北村宏司,2:00.4,クビ,306.7,11,430(0),[東] 伊藤大士
5,6,6,12,エラン,牡3,57,ピーヒュ,2:00.5,3/4,66.9,9,452(+2),[東] 蛯名正義
6,7,1,1,ジーティーオウジャ,牡3,57,キングス,2:00.6,クビ,34.3,8,518(+6),[東] 大竹正博
7,8,3,5,レディマキシマ,牝3,55,丹内祐次,2:00.7,1/2,20.6,7,480(-6),[東] 中野栄治
8,9,7,13,イモータルバード,牡3,57,津村明秀,2:01.0,2,19.8,6,476(-2),[東] 栗田徹
9,10,3,6,フェスティヴカレン,牝3,55,江田照男,2:01.3,1.3/4,489.1,15,448(-6),[東] 天間昭一


上記で、１つのレースの結果は取得できた<br>
ここからは１年間の全レースの取得を目指す<br>
URLの最後の12桁がレースidになっているので<br>
レースidを変更しながら取得していく

## 開催日一覧を取得

In [13]:
url = "https://race.netkeiba.com/top/calendar.html?year=2024&month=1"

headers = {"User-Agent": "Mozilla/5.0"}
request = Request(url, headers=headers)
html = urlopen(request).read()
html

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n<!-- block=common__meta_tag_common_race_schedule (d) -->\n<meta http-equiv="content-language" content="ja">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n<meta name="viewport" content="width=1000">\n<meta name="format-detection" content="telephone=no" />\n<title>\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8 | \xa5\xec\xa1\xbc\xa5\xb9\xbe\xf0\xca\xf3(JRA) - netkeiba</title>\n<meta name="description" content="\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8\xa5\xda\xa1\xbc\xa5\xb8\xa4\xc7\xa4\xb9\xa1\xa3\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xe4\xba\xc7\xbf\xb7\xa5\xaa\xa5\xc3\xa5\xba\xa1\xa2\xa5\xec\xa1\xbc\xa5\xb9\xb7\xeb\xb2\xcc\xc2\xae\xca\xf3\xa1\xa2\xca\xa7\xcc\xe1\xbe\xf0\xca\xf3\xa4\xf2\xa4\xcf\xa4\xb8\xa4\xe1\xa1\xa2\xb6\xa5\xc7\xcf\xcd\xbd\xc1\xdb\xa4\

レースが存在する日は、htmlのaタグにレース日付が入っていそう<br>
BeutifulSoupでhtmlを取得・解析・編集する<br>

BeautifulSoupもインストール必要<br>
<span style="color: red;">
必ず「pip install beautifulsoup4」というようにバージョン指定すること！<br>
しっかり記載しないと「BeautifulSoup3がインストールされ、python3では使えない</span>

In [14]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_common_race_schedule (d) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>開催日程 | レース情報(JRA) - netkeiba</title>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,開催日程,スケジュール,開催日,netkeiba,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba_logo_keiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="開催日程 | レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/top/calendar.html" property="og:url"/>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最

ページ全体のhtmlが取得されているので、カレンダー部分のみ抜き出す

In [15]:
#findは対象のタグが複数ある場合、最初の要素を取得する
soup.find("table")

<table class="Calendar_Table">
<tbody>
<tr class="Header">
<th>月</th>
<th>火</th>
<th>水</th>
<th>木</th>
<th>金</th>
<th class="Sat_Cell">土</th>
<th class="Sun_Cell">日</th>
</tr>
<tr class="Week">
<td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">1</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">2</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">3</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">4</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">5</span></p>
</div>
</td><td class="RaceCellBox">
<a href="../top/race_list.html?kaisai_date=20240106" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">6</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">京都</span><span class="JName">京都金杯</span><

In [16]:
#念のため、tableタグのclass名を指定する
soup.find("table", class_="Calendar_Table")

<table class="Calendar_Table">
<tbody>
<tr class="Header">
<th>月</th>
<th>火</th>
<th>水</th>
<th>木</th>
<th>金</th>
<th class="Sat_Cell">土</th>
<th class="Sun_Cell">日</th>
</tr>
<tr class="Week">
<td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">1</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">2</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">3</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">4</span></p>
</div>
</td><td class="RaceCellBox">
<div class="RaceKaisaiBox">
<p><span class="Day">5</span></p>
</div>
</td><td class="RaceCellBox">
<a href="../top/race_list.html?kaisai_date=20240106" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">6</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">京都</span><span class="JName">京都金杯</span><

In [17]:
#今度は aタグを抜き出す
soup.find("table", class_="Calendar_Table").find("a")

<a href="../top/race_list.html?kaisai_date=20240106" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">6</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">京都</span><span class="JName">京都金杯</span></p>
</div><!-- /. RaceKaisaiBox-->
</a>

In [18]:
#一旦、aという変数に入れておく
a = soup.find("table", class_="Calendar_Table").find("a")
a

<a href="../top/race_list.html?kaisai_date=20240106" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">6</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">京都</span><span class="JName">京都金杯</span></p>
</div><!-- /. RaceKaisaiBox-->
</a>

In [19]:
#aタグの中のhrefを抜き出す
a["href"]

'../top/race_list.html?kaisai_date=20240106'

In [20]:
#kaisai_dateの値のみ取り出したいので正規表現を扱うreを使用する
import re
#バックスラッシュ(\)d{8}で、8桁の数字の繰り返しを表す
#Macでバックスラッシュは、option + ¥ 
re.findall(r"kaisai_date=\d{8}", a["href"])

['kaisai_date=20240106']

In [21]:
#上から数字の部分だけを取り出す→「\d{8}」の前後を()で囲う
re.findall(r"kaisai_date=(\d{8})", a["href"])

['20240106']

In [22]:
#上ではリスト型なので最初の要素だけ取り出す
re.findall(r"kaisai_date=(\d{8})", a["href"])[0]

'20240106'

In [23]:
#他の日付も取り出してみる
soup.find("table", class_="Calendar_Table").find_all("a")

[<a href="../top/race_list.html?kaisai_date=20240106" target="_parent" tile="">
 <div class="RaceKaisaiBox HaveData">
 <p><span class="Day">6</span></p>
 <p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
 <p><span class="JyoName">京都</span><span class="JName">京都金杯</span></p>
 </div><!-- /. RaceKaisaiBox-->
 </a>,
 <a href="../top/race_list.html?kaisai_date=20240107" target="_parent" tile="">
 <div class="RaceKaisaiBox HaveData">
 <p><span class="Day">7</span></p>
 <p><span class="JyoName">中山</span><span class="JName">フェアリー</span></p>
 <p><span class="JyoName">京都</span></p>
 </div><!-- /. RaceKaisaiBox-->
 </a>,
 <a href="../top/race_list.html?kaisai_date=20240108" target="_parent" tile="">
 <div class="RaceKaisaiBox HaveData">
 <p><span class="Day">8</span></p>
 <p><span class="JyoName">中山</span></p><p><span class="JyoName">京都</span><span class="JName">シンザン記</span></p>
 </div><!-- /. RaceKaisaiBox-->
 </a>,
 <a href="../top/race_list.html?kaisai_date=20240113" target

In [24]:
#リスト型で取得されるので、リストに代入する
a_list = soup.find("table", class_="Calendar_Table").find_all("a")

In [25]:
#1月の開催日をすべて表示してみる
for a in a_list:
    print(re.findall(r"kaisai_date=(\d{8})", a["href"])[0])

20240106
20240107
20240108
20240113
20240114
20240120
20240121
20240127
20240128


In [78]:
#1月の開催日をリストに入れる
kaisai_date_list = []
for a in a_list:
    kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
    kaisai_date_list.append(kaisai_date)

In [79]:
kaisai_date_list

['20240106',
 '20240107',
 '20240108',
 '20240113',
 '20240114',
 '20240120',
 '20240121',
 '20240127',
 '20240128']

In [28]:
#他の月も取得できるようにする
#url = "https://race.netkeiba.com/top/calendar.html?year=2024&month=1"の
# monthを可変にする関数を作る
def scrape_kaisai_date(year, month):
    # f文字列を使うと{}の中身に変数を指定できる
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    print(url)

In [29]:
# 例えば
scrape_kaisai_date(year=2024, month=1)

https://race.netkeiba.com/top/calendar.html?year=2024&month=1


In [80]:
#これまでの内容を関数にまとめていく
def scrape_kaisai_date(year, month):
    #スクレイピング
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    headers = {"User-Agent": "Mozilla/5.0"}
    request = Request(url, headers=headers)
    html = urlopen(request).read()
    #htmlを加工
    soup = BeautifulSoup(html)
    a_list = soup.find("table", class_="Calendar_Table").find_all("a")
    kaisai_date_list = []
    for a in a_list:
        kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
        kaisai_date_list.append(kaisai_date)
    
    return kaisai_date_list
    
    

In [31]:
scrape_kaisai_date(year=2024, month=1)

['20240106',
 '20240107',
 '20240108',
 '20240113',
 '20240114',
 '20240120',
 '20240121',
 '20240127',
 '20240128']

In [32]:
#こういう形で、日付の範囲していをしたい
scrape_kaisai_date(from_="2024-01", to_="2024-12")

TypeError: scrape_kaisai_date() got an unexpected keyword argument 'from_'

In [33]:
#一旦、fromとtoを変数にいれる
from_="2024-01"
to_="2024-12"

In [34]:
#pandasのdate_rangeを使う
#freqに指定した"M"は月ごと、"S"はスタート
pd.date_range(from_, to_, freq="MS")

DatetimeIndex(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
              dtype='datetime64[ns]', freq='MS')

In [35]:
#MEにすれば月末日が取れる
pd.date_range(from_, to_, freq="ME")

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31', '2024-08-31',
               '2024-09-30', '2024-10-31', '2024-11-30'],
              dtype='datetime64[ns]', freq='ME')

In [36]:
#日付の一覧をforで回してみる
for date in pd.date_range(from_, to_, freq="MS"):
    print(date)

2024-01-01 00:00:00
2024-02-01 00:00:00
2024-03-01 00:00:00
2024-04-01 00:00:00
2024-05-01 00:00:00
2024-06-01 00:00:00
2024-07-01 00:00:00
2024-08-01 00:00:00
2024-09-01 00:00:00
2024-10-01 00:00:00
2024-11-01 00:00:00
2024-12-01 00:00:00


In [37]:
#ちなみに年だけや月だけも取得可能
for date in pd.date_range(from_, to_, freq="MS"):
    print(date.month)

1
2
3
4
5
6
7
8
9
10
11
12


In [38]:
#これを利用して、年月を指定したURLを作ってみる
for date in pd.date_range(from_, to_, freq="MS"):
    year = date.year
    month = date.month
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    print(url)

https://race.netkeiba.com/top/calendar.html?year=2024&month=1
https://race.netkeiba.com/top/calendar.html?year=2024&month=2
https://race.netkeiba.com/top/calendar.html?year=2024&month=3
https://race.netkeiba.com/top/calendar.html?year=2024&month=4
https://race.netkeiba.com/top/calendar.html?year=2024&month=5
https://race.netkeiba.com/top/calendar.html?year=2024&month=6
https://race.netkeiba.com/top/calendar.html?year=2024&month=7
https://race.netkeiba.com/top/calendar.html?year=2024&month=8
https://race.netkeiba.com/top/calendar.html?year=2024&month=9
https://race.netkeiba.com/top/calendar.html?year=2024&month=10
https://race.netkeiba.com/top/calendar.html?year=2024&month=11
https://race.netkeiba.com/top/calendar.html?year=2024&month=12


In [81]:
#これを利用して関数を改良する
import time #sleep用

def scrape_kaisai_date(from_, to_):
    kaisai_date_list = []   #配列の初期化はforの外へ
    
    for date in pd.date_range(from_, to_, freq="MS"):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        headers = {"User-Agent": "Mozilla/5.0"}
        request = Request(url, headers=headers)
        html = urlopen(request).read()  #スクレイピング
        #forでスクレイピングする場合サイトに負荷をかけないように必ずsleepを入れる！！
        time.sleep(1)   # 1秒スリープ
        #htmlを加工
        soup = BeautifulSoup(html)
        a_list = soup.find("table", class_="Calendar_Table").find_all("a") 
        for a in a_list:
            kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_date_list.append(kaisai_date)
    
    return kaisai_date_list

In [40]:
scrape_kaisai_date(from_="2024-01", to_="2024-12")

['20240106',
 '20240107',
 '20240108',
 '20240113',
 '20240114',
 '20240120',
 '20240121',
 '20240127',
 '20240128',
 '20240203',
 '20240204',
 '20240210',
 '20240211',
 '20240217',
 '20240218',
 '20240224',
 '20240225',
 '20240302',
 '20240303',
 '20240309',
 '20240310',
 '20240316',
 '20240317',
 '20240323',
 '20240324',
 '20240330',
 '20240331',
 '20240406',
 '20240407',
 '20240413',
 '20240414',
 '20240420',
 '20240421',
 '20240427',
 '20240428',
 '20240504',
 '20240505',
 '20240511',
 '20240512',
 '20240518',
 '20240519',
 '20240525',
 '20240526',
 '20240601',
 '20240602',
 '20240608',
 '20240609',
 '20240615',
 '20240616',
 '20240622',
 '20240623',
 '20240629',
 '20240630',
 '20240706',
 '20240707',
 '20240713',
 '20240714',
 '20240720',
 '20240721',
 '20240727',
 '20240728',
 '20240803',
 '20240804',
 '20240810',
 '20240811',
 '20240817',
 '20240818',
 '20240824',
 '20240825',
 '20240831',
 '20240901',
 '20240907',
 '20240908',
 '20240914',
 '20240915',
 '20240916',
 '20240921',

待ち時間があるので、プログレスバーを表示する

In [41]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [82]:
#実行に待ち時間があるので、
import time #sleep用
from tqdm.notebook import tqdm

def scrape_kaisai_date(from_, to_):
    kaisai_date_list = []   #配列の初期化はforの外へ
    
    #tqdmをfor文のin のあとにつける
    for date in tqdm(pd.date_range(from_, to_, freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        headers = {"User-Agent": "Mozilla/5.0"}
        request = Request(url, headers=headers)
        html = urlopen(request).read()  #スクレイピング
        #forでスクレイピングする場合サイトに負荷をかけないように必ずsleepを入れる！！
        time.sleep(1)   # 1秒スリープ
        #htmlを加工
        soup = BeautifulSoup(html)
        a_list = soup.find("table", class_="Calendar_Table").find_all("a") 
        for a in a_list:
            kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_date_list.append(kaisai_date)
    
    return kaisai_date_list


In [44]:
scrape_kaisai_date(from_="2024-01", to_="2024-12")

  0%|          | 0/12 [00:00<?, ?it/s]

['20240106',
 '20240107',
 '20240108',
 '20240113',
 '20240114',
 '20240120',
 '20240121',
 '20240127',
 '20240128',
 '20240203',
 '20240204',
 '20240210',
 '20240211',
 '20240217',
 '20240218',
 '20240224',
 '20240225',
 '20240302',
 '20240303',
 '20240309',
 '20240310',
 '20240316',
 '20240317',
 '20240323',
 '20240324',
 '20240330',
 '20240331',
 '20240406',
 '20240407',
 '20240413',
 '20240414',
 '20240420',
 '20240421',
 '20240427',
 '20240428',
 '20240504',
 '20240505',
 '20240511',
 '20240512',
 '20240518',
 '20240519',
 '20240525',
 '20240526',
 '20240601',
 '20240602',
 '20240608',
 '20240609',
 '20240615',
 '20240616',
 '20240622',
 '20240623',
 '20240629',
 '20240630',
 '20240706',
 '20240707',
 '20240713',
 '20240714',
 '20240720',
 '20240721',
 '20240727',
 '20240728',
 '20240803',
 '20240804',
 '20240810',
 '20240811',
 '20240817',
 '20240818',
 '20240824',
 '20240825',
 '20240831',
 '20240901',
 '20240907',
 '20240908',
 '20240914',
 '20240915',
 '20240916',
 '20240921',

ここまでで、ステップ１の開催日一覧を取得できた
この内容をスクリプトにまとめていく
src/scraiping.pyにまとめる

## 開催ページからレースidを取得

In [45]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20240106"

In [46]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [47]:
headers = {"User-Agent": "Mozilla/5.0"}
request = Request(url, headers=headers)
html = urlopen(request).read()
soup = BeautifulSoup(html)

In [48]:
#ページからレースidを探す
soup.find("div", calss_="RaceList_Box")

上記の処理では何も取得できない<br>
JavaScriptで動的に生成されているhtmlの場合<br>
BeautifulSoupで取得できないことがある<br>
取得できないので、ChromeDriverを使う<br>
ChromeDriverで、動的なページを静的なページのように開いて<br>
スクレイピングする

In [49]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [51]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

#seleniumとGoogle Chromのバージョンを合わせる必要がある
#これを書くと自動でバージョンを合わせて、インストールしたpathを返してくれる
driver_path = ChromeDriverManager().install()
driver_path

/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'/Users/matsukawanaoya/.wdm/drivers/chromedriver/mac64/133.0.6943.126/chromedriver-mac-arm64/chromedriver'

In [52]:
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--user-agent=Mozilla/5.0")
# これでChromeが起動される
driver = webdriver.Chrome(service=Service(driver_path), options=options)

#これを使えば、Webページの自動操作なども可能になるらしい

In [53]:
# 起動したChromeに先程のurlを表示する
driver.get(url)

In [54]:
from selenium.webdriver.common.by import By

# find_elementも、対象が複数あれば先頭のみ
driver.find_element(By.CLASS_NAME, "RaceList_Box")

<selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.62")>

In [55]:
from selenium.webdriver.common.by import By

#こちらは対象すべて
driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")

[<selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.129")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.130")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.131")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.132")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.133")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e23

In [56]:
#上の情報をリストに入れる
li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.129")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.130")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.131")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.132")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.133")>,
 <selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e23

In [57]:
li = li_list[0]
li

<selenium.webdriver.remote.webelement.WebElement (session="64471e933b3a2fe64e2343780248ab47", element="f.18E40B85044F27C8DB9A2DE2EE92C3AD.d.4866DA1AE3DEDFB768311A8E1A2EC12A.e.129")>

In [58]:
href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
href

'https://race.netkeiba.com/race/result.html?race_id=202406010101&rf=race_list'

In [59]:
#urlからidの部分を取り出す
import re

re.findall(r"race_id=(\d{12})", href)[0]

'202406010101'

In [60]:
for li in li_list:
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    print(race_id)

202406010101
202406010102
202406010103
202406010104
202406010105
202406010106
202406010107
202406010108
202406010109
202406010110
202406010111
202406010112
202408010101
202408010102
202408010103
202408010104
202408010105
202408010106
202408010107
202408010108
202408010109
202408010110
202408010111
202408010112


In [61]:
#取得したidを配列に格納する
race_id_list = []
for li in li_list:
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    race_id_list.append(race_id)

In [62]:
len(race_id_list)

24

In [63]:
#ChromeDriverの終了
driver.quit()

## ステップの１と２をつなげて、2024年の全レースidを取得する

### 関数化

In [64]:
import scraping

In [9]:
kaisai_date_list = scraping.scrape_kaisai_date(from_="2024-01", to_="2024-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
#これはめちゃくちゃ時間かかる版
import time

def scrape_race_id_list(kaisai_date_list: list[str]):
    driver_path = ChromeDriverManager().install()
    options = Options()
    options.add_argument("--user-agent=Mozilla/5.0")
    driver = webdriver.Chrome(service=Service(driver_path), options=options)
    race_id_list = []
    for kaisai_date in kaisai_date_list:
        url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
        driver.get(url)
        time.sleep(1)
        li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
        for li in li_list:
            href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
            race_id = re.findall(r"race_id=(\d{12})", href)[0]
            race_id_list.append(race_id)
    driver.quit()
    return race_id_list

In [ ]:
#このまま関数を実行すると、Chromeが開いて処理が実行されるが
#めちゃくちゃ時間がかかってしまう
scrape_race_id_list(kaisai_date_list)

In [ ]:
#optionを設定と進捗を追加する
import time
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

def scrape_race_id_list(kaisai_date_list: list[str]):
    driver_path = ChromeDriverManager().install()
    options = Options()
    options.add_argument("--user-agent=Mozilla/5.0")
    # ヘッドレスモードを指定することで、バックグランドで実行される
    options.add_argument("--headless")
    race_id_list = []
    # バックグランド実行にしたことでChromeが起動されなくなるので、下の構文を変更する
    # こうすることで、処理が終了すると自動でdriverをquitしてくれる
    with webdriver.Chrome(service=Service(driver_path), options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            driver.get(url)
            time.sleep(1)
            li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
            for li in li_list:
                href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
                race_id = re.findall(r"race_id=(\d{12})", href)[0]
                race_id_list.append(race_id)
    return race_id_list

In [ ]:
# 少し速くなったけど、エラー時の処理を追加する
scrape_race_id_list(kaisai_date_list)

In [83]:
#tryを追加する
import time
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
import traceback    # 追加

def scrape_race_id_list(kaisai_date_list: list[str]):
    driver_path = ChromeDriverManager().install()
    options = Options()
    options.add_argument("--user-agent=Mozilla/5.0")
    options.add_argument("--headless")
    race_id_list = []
    with webdriver.Chrome(service=Service(driver_path), options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                # エラーの内容を表示する
                print(traceback.format_exc())
                # 処理に失敗したとき、forをbreakすることで途中までの結果がreturnされる
                break
    return race_id_list

In [77]:
# この処理を途中で止めると、止まった時点のurlとlistの途中までは取得できる
scrape_race_id_list(kaisai_date_list)

  0%|          | 0/9 [00:00<?, ?it/s]

stopped at https://race.netkeiba.com/top/race_list.html?kaisai_date=20240106
Traceback (most recent call last):
  File "/var/folders/4j/53myqwhn0xxd2sgb7ygd10_c0000gn/T/ipykernel_3115/4142642515.py", line 17, in scrape_race_id_list
    driver.get(url)
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 454, in get
    self.execute(Command.GET, {"url": url})
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 427, in execute
    response = self.command_executor.execute(driver_command, params)
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/remote_connection.py", line 404, in execute
    return self._request(command_info[0], url, body=data)
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remo

[]

上記の状態で、関数をscraping.pyに移動する

### スクリプトのチェック

In [84]:
import scraping

In [85]:
# 先頭の10件のみ取得
race_id_list = scraping.scrape_race_id_list(kaisai_date_list[:10])

AttributeError: module 'scraping' has no attribute 'scrape_race_id_list'

notebookにおけるモジュールのリロード<br>
追加したscrape_race_id_listが読み込まれていない<br>
notebookを再起動する必要があるが、これまで実行した処理や変数もすべて初期化されてしまう

In [4]:
import scraping
%load_ext autoreload    
# %が付いたものをマジックコマンドという

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# スクリプトの中身が変更された場合
# これを実行すれば、%load_ext autoreloadと一緒にimportしたスクリプトが
# 再起動をしなくても更新されるようになる
# 今回はすでにimportしてからload_extをつけたので再起動する必要がある
%autoreload

In [ ]:
# 再度、先頭の10日分を読み込んでみる
race_id_list = scraping.scrape_race_id_list(kaisai_date_list[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
race_id_list

['202406010101',
 '202406010102',
 '202406010103',
 '202406010104',
 '202406010105',
 '202406010106',
 '202406010107',
 '202406010108',
 '202406010109',
 '202406010110',
 '202406010111',
 '202406010112',
 '202408010101',
 '202408010102',
 '202408010103',
 '202408010104',
 '202408010105',
 '202408010106',
 '202408010107',
 '202408010108',
 '202408010109',
 '202408010110',
 '202408010111',
 '202408010112',
 '202406010201',
 '202406010202',
 '202406010203',
 '202406010204',
 '202406010205',
 '202406010206',
 '202406010207',
 '202406010208',
 '202406010209',
 '202406010210',
 '202406010211',
 '202406010212',
 '202408010201',
 '202408010202',
 '202408010203',
 '202408010204',
 '202408010205',
 '202408010206',
 '202408010207',
 '202408010208',
 '202408010209',
 '202408010210',
 '202408010211',
 '202408010212',
 '202406010301',
 '202406010302',
 '202406010303',
 '202406010304',
 '202406010305',
 '202406010306',
 '202406010307',
 '202406010308',
 '202406010309',
 '202406010310',
 '202406010311

### レースidから、レース結果を取得する

In [ ]:
from urllib.request import urlopen, Request
# レース結果ページのURL
url = "https://db.netkeiba.com/race/202406010105/"
headers = {"User-Agent": "Mozilla/5.0"}
request = Request(url, headers=headers)
html = urlopen(request).read() 

In [18]:
import pandas as pd

pd.read_html(html)

[    着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気  \
 0     1    8   15   ミッキーラッキー  牡3  57   キング  1:59.8    NaN    2.8    1   
 1     2    1    2  マイネルモメンタム  牡3  57  石川裕紀  1:59.8    アタマ    7.3    4   
 2     3    4    7  ウイントレメンデス  牡3  57  横山武史  1:59.8     ハナ    4.4    3   
 3     4    5   10  ウォータースケイプ  牡3  57  戸崎圭太  2:00.3      3    4.2    2   
 4     5    8   16      ハイラント  牡3  57  北村宏司  2:00.4     クビ  306.7   11   
 5     6    6   12        エラン  牡3  57  ピーヒュ  2:00.5    3/4   66.9    9   
 6     7    1    1  ジーティーオウジャ  牡3  57  キングス  2:00.6     クビ   34.3    8   
 7     8    3    5    レディマキシマ  牝3  55  丹内祐次  2:00.7    1/2   20.6    7   
 8     9    7   13   イモータルバード  牡3  57  津村明秀  2:01.0      2   19.8    6   
 9    10    3    6  フェスティヴカレン  牝3  55  江田照男  2:01.3  1.3/4  489.1   15   
 10   11    2    4      ネクタール  牝3  52  佐藤翔馬  2:01.3     クビ  501.7   16   
 11   12    2    3   マイネルガンナー  牡3  57  柴田大知  2:01.3     クビ  162.1   10   
 12   13    7   14   フォローウィンド  牡3  57 

In [19]:
pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,8,15,ミッキーラッキー,牡3,57,キング,1:59.8,NaN,2.8,1,550(+4),[東] 堀宣行
1,2,1,2,マイネルモメンタム,牡3,57,石川裕紀,1:59.8,アタマ,7.3,4,470(+2),[東] 相沢郁
2,3,4,7,ウイントレメンデス,牡3,57,横山武史,1:59.8,ハナ,4.4,3,480(+4),[東] 鈴木伸尋
3,4,5,10,ウォータースケイプ,牡3,57,戸崎圭太,2:00.3,3,4.2,2,554(-2),[東] 宮田敬介
4,5,8,16,ハイラント,牡3,57,北村宏司,2:00.4,クビ,306.7,11,430(0),[東] 伊藤大士
5,6,6,12,エラン,牡3,57,ピーヒュ,2:00.5,3/4,66.9,9,452(+2),[東] 蛯名正義
6,7,1,1,ジーティーオウジャ,牡3,57,キングス,2:00.6,クビ,34.3,8,518(+6),[東] 大竹正博
7,8,3,5,レディマキシマ,牝3,55,丹内祐次,2:00.7,1/2,20.6,7,480(-6),[東] 中野栄治
8,9,7,13,イモータルバード,牡3,57,津村明秀,2:01.0,2,19.8,6,476(-2),[東] 栗田徹
9,10,3,6,フェスティヴカレン,牝3,55,江田照男,2:01.3,1.3/4,489.1,15,448(-6),[東] 天間昭一


レース結果ページURLの、レースidを取得したリストで切り替えれば<br>
全レースの結果が取得できる

In [20]:
import pickle

with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [22]:
len(race_id_list)

3454

レース結果ページからは複数のtableの情報を必要とするので<br>
html自体を保存しておく

In [ ]:
# フォルダ作成
!mkdir ../data

# macでは以下のパス指定で動くがWindowsでは動かないかも
with open("../data/html/202406010105.bin", "wb") as f:
    f.write(html)

In [25]:
# 環境に左右されないように、パスを指定するときはpathlibを使う
from pathlib import Path

HTML_DIR = Path("..", "data", "html")
HTML_DIR

PosixPath('../data/html')

In [26]:
# pathlibには、直接ディレクトリをつなげていくことができる
HTML_DIR / "202406010105.bin"

PosixPath('../data/html/202406010105.bin')

In [ ]:
# よって、こうなる
with open(HTML_DIR / "202406010105.bin", "wb") as f:
    f.write(html)

In [ ]:
# スクレイピングした結果をファイルに保存
import time

for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    headers = {"User-Agent": "Mozilla/5.0"}
    request = Request(url, headers=headers)
    html = urlopen(request).read()
    time.sleep(1)
    with open(HTML_DIR / f"{race_id}.bin", "wb") as f:
        f.write(html)

In [35]:
# 関数化
from tqdm.notebook import tqdm

def scrape_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list[:10]):
        url = f"https://db.netkeiba.com/race/{race_id}"
        headers = {"User-Agent": "Mozilla/5.0"}
        request = Request(url, headers=headers)
        html = urlopen(request).read()
        time.sleep(1)
        filepath = save_dir / f"{race_id}.bin"
        with open(filepath, "wb") as f:
            f.write(html)
        html_path_list.append(filepath)
    return html_path_list

In [36]:
html_path_list = scrape_html_race(race_id_list[:10], save_dir=HTML_DIR)

  0%|          | 0/10 [00:00<?, ?it/s]

In [34]:
html_path_list

[PosixPath('../data/html/202406010101.bin'),
 PosixPath('../data/html/202406010102.bin'),
 PosixPath('../data/html/202406010103.bin'),
 PosixPath('../data/html/202406010104.bin'),
 PosixPath('../data/html/202406010105.bin'),
 PosixPath('../data/html/202406010106.bin'),
 PosixPath('../data/html/202406010107.bin'),
 PosixPath('../data/html/202406010108.bin'),
 PosixPath('../data/html/202406010109.bin'),
 PosixPath('../data/html/202406010110.bin')]

In [ ]:
# すでに同じ名前のファイルが存在する場合、スキップさせる方法

# is_file()で存在する場合はTrueを返してくれる
html_path_list[0].is_file()

True

In [38]:
# 存在しないファイルの場合はFalse
(HTML_DIR / "ddddd.bin").is_file()

False

In [45]:
# 改良版
from tqdm.notebook import tqdm

def scrape_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin"  # ファイルパスをスクレイピング前に移動
        # ファイルが既に存在する場合はスキップする
        if filepath.is_file():
            print(f"skipped: {race_id}")
        else:
            url = f"https://db.netkeiba.com/race/{race_id}"
            headers = {"User-Agent": "Mozilla/5.0"}
            request = Request(url, headers=headers)
            html = urlopen(request).read()
            time.sleep(1)
            with open(filepath, "wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [ ]:
html_path_list = scrape_html_race(race_id_list[:12], save_dir=HTML_DIR)

関数ができたので、scraping.pyに移動する

次の工程では、上記のpathリストのファイルを読み込んで<br>
htmlとして加工していく

In [48]:
HTML_RACE_DIR = HTML_DIR / "race"
HTML_RACE_DIR

PosixPath('../data/html/race')

In [52]:
# globで指定ディレクトリのファイルを取得できる
# globはgenerator型で返すのでリスト型にする
html_paths_race =  list(HTML_RACE_DIR.glob("*"))
html_paths_race

[PosixPath('../data/html/race/202408060405.bin'),
 PosixPath('../data/html/race/202409020308.bin'),
 PosixPath('../data/html/race/202402010807.bin'),
 PosixPath('../data/html/race/202408070602.bin'),
 PosixPath('../data/html/race/202408060411.bin'),
 PosixPath('../data/html/race/202408031110.bin'),
 PosixPath('../data/html/race/202405050506.bin'),
 PosixPath('../data/html/race/202403020409.bin'),
 PosixPath('../data/html/race/202408031104.bin'),
 PosixPath('../data/html/race/202405050512.bin'),
 PosixPath('../data/html/race/202405040701.bin'),
 PosixPath('../data/html/race/202401020502.bin'),
 PosixPath('../data/html/race/202409010208.bin'),
 PosixPath('../data/html/race/202408050505.bin'),
 PosixPath('../data/html/race/202407030312.bin'),
 PosixPath('../data/html/race/202407020101.bin'),
 PosixPath('../data/html/race/202402010608.bin'),
 PosixPath('../data/html/race/202406050903.bin'),
 PosixPath('../data/html/race/202407030306.bin'),
 PosixPath('../data/html/race/202408050511.bin'),


In [53]:
len(html_paths_race)

3444

In [ ]:
for html_path in html_paths_race:
    with open(html_path, "rb") as f:
        html = f.read()
        # htmlが読み込まれていることを確認したいので、１件だけprintして
        # breakでforを抜ける
        print(html)
        break

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>2\xba\xd0\xbf\xb7\xc7\xcf\xa1\xc32024\xc7\xaf11\xb7\xee10\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\xa6\xc

In [ ]:
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        html = f.read()
        # データフレームを取得　データフレームは行と列で構成される２次元のデータ構造
        df = pd.read_html(html)[0]
        print(df)

   着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気     馬体重  \
0    1    8   10    エコロディノス  牡2  56   ムーア  2:00.9    NaN    2.3    1  472(0)   
1    2    5    5    トロワヴァレー  牡2  56  Ｃ．デム  2:01.5  3.1/2    3.9    2  470(0)   
2    3    1    1      コジオスコ  牡2  56  ルメール  2:01.9  2.1/2    5.3    3  494(0)   
3    4    7    8    アロマフェリス  牝2  55  鮫島克駿  2:02.4      3   30.2    7  408(0)   
4    5    8    9  サウスオブロンドン  牡2  55  泉谷楓真  2:02.7      2  107.7    9  478(0)   
5    6    6    6  ロードアルディート  牡2  56  松山弘平  2:02.8    3/4    6.8    4  450(0)   
6    7    4    4  ヤマニンソレアード  牝2  55  古川吉洋  2:02.9     クビ   29.6    6  462(0)   
7    8    3    3  ジューンノールック  牝2  55  坂井瑠星  2:03.0    1/2    7.6    5  502(0)   
8    9    2    2     アンミツヒメ  牝2  55   浜中俊  2:03.0     クビ   61.8    8  446(0)   
9   10    7    7   タイセイルシエル  牡2  56   酒井学  2:04.3      8  166.9   10  432(0)   

        調教師  
0  [西] 大久保龍  
1  [西] 吉岡辰弥  
2  [西] 武幸四郎  
3  [西] 辻野泰之  
4  [西] 松永幹夫  
5  [西] 杉山晴紀  
6  [西] 四位洋文  
7   [西]

In [ ]:

dfs = {}
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        # stem（ステム）でパスのレースidを取り出せる
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        dfs[race_id] = df

In [57]:
dfs

{'202408060405':    着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気     馬体重  \
 0    1    8   10    エコロディノス  牡2  56   ムーア  2:00.9    NaN    2.3    1  472(0)   
 1    2    5    5    トロワヴァレー  牡2  56  Ｃ．デム  2:01.5  3.1/2    3.9    2  470(0)   
 2    3    1    1      コジオスコ  牡2  56  ルメール  2:01.9  2.1/2    5.3    3  494(0)   
 3    4    7    8    アロマフェリス  牝2  55  鮫島克駿  2:02.4      3   30.2    7  408(0)   
 4    5    8    9  サウスオブロンドン  牡2  55  泉谷楓真  2:02.7      2  107.7    9  478(0)   
 5    6    6    6  ロードアルディート  牡2  56  松山弘平  2:02.8    3/4    6.8    4  450(0)   
 6    7    4    4  ヤマニンソレアード  牝2  55  古川吉洋  2:02.9     クビ   29.6    6  462(0)   
 7    8    3    3  ジューンノールック  牝2  55  坂井瑠星  2:03.0    1/2    7.6    5  502(0)   
 8    9    2    2     アンミツヒメ  牝2  55   浜中俊  2:03.0     クビ   61.8    8  446(0)   
 9   10    7    7   タイセイルシエル  牡2  56   酒井学  2:04.3      8  166.9   10  432(0)   
 
         調教師  
 0  [西] 大久保龍  
 1  [西] 吉岡辰弥  
 2  [西] 武幸四郎  
 3  [西] 辻野泰之  
 4  [西] 松永幹夫  
 

In [ ]:
# レースidを指定して抜き出すこともできる
dfs['202402010807']

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,5,5,エンセリオ,牝3,53,鮫島克駿,1:46.1,NaN,8.3,5,442(0),[東] 小笠倫弘
1,2,8,8,メジャーデビュー,牝3,53,丹内祐次,1:46.3,1.1/4,2.9,2,436(+6),[東] 鈴木慎太
2,3,1,1,ポワンキュルミナン,牡4,58,菱田裕二,1:46.7,2.1/2,34.0,7,426(0),[西] 岡田稲男
3,4,6,6,ゼンダンスカイ,牡3,52,小林勝太,1:46.7,アタマ,2.7,1,456(+2),[西] 角田晃一
4,5,7,7,ベイビールビオ,牡5,58,佐々木大,1:47.0,1.3/4,16.9,6,500(0),[東] 田村康仁
5,6,2,2,メイショウクモイ,牡4,58,武豊,1:47.8,5,5.4,3,476(-4),[西] 飯田祐史
6,7,4,4,ワンダージュノ,セ5,58,黛弘人,1:48.4,3.1/2,123.0,8,478(-10),[西] 森田直行
7,8,3,3,ピースヒロフェイス,牝3,53,古川吉洋,1:48.8,2.1/2,6.9,4,460(-4),[東] 牧光二


In [59]:
#このままだと扱いづらいので、全部をつなげる
pd.concat(dfs.values())

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,8,10,エコロディノス,牡2,56,ムーア,2:00.9,NaN,2.3,1,472(0),[西] 大久保龍
1,2,5,5,トロワヴァレー,牡2,56,Ｃ．デム,2:01.5,3.1/2,3.9,2,470(0),[西] 吉岡辰弥
2,3,1,1,コジオスコ,牡2,56,ルメール,2:01.9,2.1/2,5.3,3,494(0),[西] 武幸四郎
3,4,7,8,アロマフェリス,牝2,55,鮫島克駿,2:02.4,3,30.2,7,408(0),[西] 辻野泰之
4,5,8,9,サウスオブロンドン,牡2,55,泉谷楓真,2:02.7,2,107.7,9,478(0),[西] 松永幹夫
5,6,6,6,ロードアルディート,牡2,56,松山弘平,2:02.8,3/4,6.8,4,450(0),[西] 杉山晴紀
6,7,4,4,ヤマニンソレアード,牝2,55,古川吉洋,2:02.9,クビ,29.6,6,462(0),[西] 四位洋文
7,8,3,3,ジューンノールック,牝2,55,坂井瑠星,2:03.0,1/2,7.6,5,502(0),[西] 武英智
8,9,2,2,アンミツヒメ,牝2,55,浜中俊,2:03.0,クビ,61.8,8,446(0),[西] 四位洋文
9,10,7,7,タイセイルシエル,牡2,56,酒井学,2:04.3,8,166.9,10,432(0),[西] 松下武士


In [60]:
# インデックスをレースidにしたい
# indexは行数分の配列にしておく必要がある
# 配列を増やす方法
['202402010807'] * 4

['202402010807', '202402010807', '202402010807', '202402010807']

In [61]:
['202402010807'] * len(df)

['202402010807',
 '202402010807',
 '202402010807',
 '202402010807',
 '202402010807',
 '202402010807',
 '202402010807',
 '202402010807']

In [ ]:
# インデックスをレースidにする
dfs = {}
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        df.index = [race_id] * len(df)  # indexにレースidを設定
        dfs[race_id] = df

In [64]:
# 再度つなげて表示してみる
pd.concat(dfs.values())

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
202408060405,1,8,10,エコロディノス,牡2,56,ムーア,2:00.9,NaN,2.3,1,472(0),[西] 大久保龍
202408060405,2,5,5,トロワヴァレー,牡2,56,Ｃ．デム,2:01.5,3.1/2,3.9,2,470(0),[西] 吉岡辰弥
202408060405,3,1,1,コジオスコ,牡2,56,ルメール,2:01.9,2.1/2,5.3,3,494(0),[西] 武幸四郎
202408060405,4,7,8,アロマフェリス,牝2,55,鮫島克駿,2:02.4,3,30.2,7,408(0),[西] 辻野泰之
202408060405,5,8,9,サウスオブロンドン,牡2,55,泉谷楓真,2:02.7,2,107.7,9,478(0),[西] 松永幹夫
202408060405,6,6,6,ロードアルディート,牡2,56,松山弘平,2:02.8,3/4,6.8,4,450(0),[西] 杉山晴紀
202408060405,7,4,4,ヤマニンソレアード,牝2,55,古川吉洋,2:02.9,クビ,29.6,6,462(0),[西] 四位洋文
202408060405,8,3,3,ジューンノールック,牝2,55,坂井瑠星,2:03.0,1/2,7.6,5,502(0),[西] 武英智
202408060405,9,2,2,アンミツヒメ,牝2,55,浜中俊,2:03.0,クビ,61.8,8,446(0),[西] 四位洋文
202408060405,10,7,7,タイセイルシエル,牡2,56,酒井学,2:04.3,8,166.9,10,432(0),[西] 松下武士


In [72]:
dfs = {}
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        df.index = [race_id] * len(df)
        dfs[race_id] = df
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"    # indexに名前をつける

In [66]:
concat_df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202408060405,1,8,10,エコロディノス,牡2,56,ムーア,2:00.9,NaN,2.3,1,472(0),[西] 大久保龍
202408060405,2,5,5,トロワヴァレー,牡2,56,Ｃ．デム,2:01.5,3.1/2,3.9,2,470(0),[西] 吉岡辰弥
202408060405,3,1,1,コジオスコ,牡2,56,ルメール,2:01.9,2.1/2,5.3,3,494(0),[西] 武幸四郎
202408060405,4,7,8,アロマフェリス,牝2,55,鮫島克駿,2:02.4,3,30.2,7,408(0),[西] 辻野泰之
202408060405,5,8,9,サウスオブロンドン,牡2,55,泉谷楓真,2:02.7,2,107.7,9,478(0),[西] 松永幹夫
202408060405,6,6,6,ロードアルディート,牡2,56,松山弘平,2:02.8,3/4,6.8,4,450(0),[西] 杉山晴紀
202408060405,7,4,4,ヤマニンソレアード,牝2,55,古川吉洋,2:02.9,クビ,29.6,6,462(0),[西] 四位洋文
202408060405,8,3,3,ジューンノールック,牝2,55,坂井瑠星,2:03.0,1/2,7.6,5,502(0),[西] 武英智
202408060405,9,2,2,アンミツヒメ,牝2,55,浜中俊,2:03.0,クビ,61.8,8,446(0),[西] 四位洋文


In [90]:
# 関数化する 今回はスクレイピングしないのでsleepは不要
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_paths_race):
        try:
            with open(html_path, "rb") as f:
                race_id = html_path.stem
                html = f.read()
                df = pd.read_html(html)[0]
                df.index = [race_id] * len(df)
                dfs[race_id] = df
        except:
            print(f"stopped at {race_id}")
    concat_df = pd.concat(dfs.values())
    concat_df.index.name = "race_id"
    return concat_df

In [91]:
results = create_results(html_path_list=html_paths_race)

  0%|          | 0/3444 [00:00<?, ?it/s]

stopped at 202410020804
stopped at 202410030601
stopped at 202404030601
stopped at 202407020801
stopped at 202403020501
stopped at 202403020504
stopped at 202410020705
stopped at 202404010605
stopped at 202404030701
stopped at 202404010604
stopped at 202410020704
stopped at 202404030704


In [92]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202408060405,1,8,10,エコロディノス,牡2,56.0,ムーア,2:00.9,NaN,2.3,1.0,472(0),[西] 大久保龍
202408060405,2,5,5,トロワヴァレー,牡2,56.0,Ｃ．デム,2:01.5,3.1/2,3.9,2.0,470(0),[西] 吉岡辰弥
202408060405,3,1,1,コジオスコ,牡2,56.0,ルメール,2:01.9,2.1/2,5.3,3.0,494(0),[西] 武幸四郎
202408060405,4,7,8,アロマフェリス,牝2,55.0,鮫島克駿,2:02.4,3,30.2,7.0,408(0),[西] 辻野泰之
202408060405,5,8,9,サウスオブロンドン,牡2,55.0,泉谷楓真,2:02.7,2,107.7,9.0,478(0),[西] 松永幹夫
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202405010107,12,3,5,グランヒマラヤ,牡4,57.0,石橋脩,1:29.1,2.1/2,6.7,3.0,504(+10),[東] 松山将樹
202405010107,13,4,8,フエキ,牝5,56.0,嶋田純次,1:29.2,クビ,14.1,7.0,524(+10),[東] 和田正一
202405010107,14,6,12,ジャーニーメモリー,牡4,56.0,原優介,1:29.2,クビ,63.2,14.0,468(+10),[東] 粕谷昌央


この関数を、create_rawdf.pyに書き出す